In [ ]:
%matplotlib inline

# KDD99 Supervised Learning

## 0. Libraries

In [ ]:
import numpy as np
import pandas as pd

-----

## 1. Data Description

**Intrinsic attributes**

These attributes are extracted from the headers' area of the network packets.

Col|Feature name  | description |	type
---|--------------|-------------|------------
1  |duration 	  |length (number of seconds) of the connection |continuous
2  |protocol_type |type of the protocol, e.g. tcp, udp, etc. |discrete
3  |service 	  |network service on the destination, e.g., http, telnet, etc. |discrete
4  |flag 	      |normal or error status of the connection. The possible status are this: SF, S0, S1, S2, S3, OTH, REJ, RSTO, RSTOS0, SH, RSTRH, SHR 	|discrete 
5  |src_bytes 	  |number of data bytes from source to destination 	|continuous
6  |dst_bytes 	  |number of data bytes from destination to source 	|continuous
7  |land 	      |1 if connection is from/to the same host/port; 0 otherwise 	|discrete
8  |wrong_fragment|sum of bad checksum packets in a connection 	|continuous
9  |urgent 	      |number of urgent packets. Urgent packets are packets with the urgent bit activated 	|continuous


**Class attribute**

The 42nd attribute is the ***class_attack*** attribute, it indicates which type of connections is each instance: normal or which attack. The values it can take are the following: *anomaly, dict, dict_simple, eject, eject-fail, ffb, ffb_clear, format, format_clear, format-fail, ftp-write, guest, imap, land, load_clear, loadmodule, multihop, perl_clear, perlmagic, phf, rootkit, spy, syslog, teardrop, warez, warezclient, warezmaster, pod, back, ip- sweep, neptune, nmap, portsweep, satan, smurf and normal*.

** Categories of class attribute **


class_attack |Category
-------|--------------
smurf| dos
neptune| dos
back| dos
teardrop| dos
pod| dos
land| dos
normal|normal
satan|probe
ipsweep|probe
portsweep|probe
nmap|probe
warezclient|r2l
guess_passwd|r2l
warezmaster|r2l
imap|r2l
ftp_write|r2l
multihop|r2l
phf|r2l
spy|r2l
buffer_overflow|u2r
rootkit|u2r
loadmodule|u2r
perl|u2r

----

## 2. Load Data

### 2.1 Loading All Data

In [ ]:
allData = pd.read_csv('../data/KDD/KDDTrain+.txt', header=None, usecols=[0,1,2,3,4,5,6,7,8,41], 
                   dtype = {"duration": 'float64',
                            "protocol_type": 'object',
                            "service": 'object',
                            "flag": 'object',
                            "src_bytes": 'float64',
                            "dst_bytes": 'float64',
                            "land": 'object',
                            "wrong_fragment": 'float64',
                            "urgent": 'float64',
                            "class_attack": 'object'})

In [ ]:
allData.columns=["duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
                 "wrong_fragment","urgent", "class_attack"]

In [ ]:
allData.protocol_type = allData.protocol_type.astype('category')
allData.service = allData.service.astype('category')
allData.flag = allData.flag.astype('category')
allData.class_attack = allData.class_attack.astype('category')

In [ ]:
allData.head()

In [ ]:
allDS = allData[['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 
         'wrong_fragment', 'urgent']]

In [ ]:
allDS.head()

In [ ]:
allLabels = pd.DataFrame(allData['class_attack'], dtype='category')

In [ ]:
allLabels["is_normal"] = np.array(allLabels.class_attack == 'normal',dtype='int')

In [ ]:
allLabels.head()

In [ ]:
allLabels.shape

-----

## 3. Data Preparation

### 3.1 Encoding categorical features

In [ ]:
# import libraries
import sklearn.preprocessing as pp

In [ ]:
attack_class = list(set(allLabels.class_attack.unique().tolist()))

In [ ]:
print attack_class

In [ ]:
lb_all_attack_class = pp.LabelBinarizer()
lb_all_attack_class.fit(attack_class)
lb_all_attack_class.transform(allLabels.class_attack).shape

In [ ]:
lb_all_attack_class.classes_.shape

In [ ]:
all_attack_class_bin = lb_all_attack_class.transform(allLabels.class_attack)

allLabels_encoded = pd.DataFrame(all_attack_class_bin, 
                                       columns = ['is_'+x for x in attack_class])

** Encoding protocol_type **

In [ ]:
protocol_type_class = list(set(allDS.protocol_type.unique().tolist()))

In [ ]:
print protocol_type_class

In [ ]:
all_protocol_type_bin = pp.label_binarize(allDS.protocol_type, 
                                      classes = protocol_type_class)
all_protocol_type_DataFrame = pd.DataFrame(all_protocol_type_bin, 
                                       columns = ['is_'+x for x in protocol_type_class])

** Encoding service **

In [ ]:
service_class = list(set(allDS.service.unique().tolist()))

In [ ]:
print service_class

In [ ]:
all_service_bin = pp.label_binarize(allDS.service, 
                                      classes = service_class)
all_service_DataFrame = pd.DataFrame(all_service_bin, 
                                       columns = ['is_'+x for x in service_class])

** Encoding flag **

In [ ]:
flag_class = list(set(allDS.flag.unique().tolist()))

In [ ]:
print flag_class

In [ ]:
all_flag_bin = pp.label_binarize(allDS.flag, 
                                    classes = flag_class)
all_flag_DataFrame = pd.DataFrame(all_flag_bin, 
                                 columns = ['is_'+x for x in flag_class])

** Concatenating all de data set **

In [ ]:
allDS_encoded = pd.concat([allDS, all_protocol_type_DataFrame, all_service_DataFrame, 
                     all_flag_DataFrame, allLabels.class_attack], axis = 1)


** Selecting only numbered features **

In [ ]:
continuousCols = ["duration","src_bytes","dst_bytes","land","wrong_fragment","urgent"] + \
            [c for c in allDS_encoded.columns if c.startswith("is_")]
allDS_encoded = pd.DataFrame(allDS_encoded[continuousCols], dtype='float64')
print allDS_encoded.shape

### 3.2 Spliting dataset

In [ ]:
from sklearn import cross_validation

In [ ]:
trainDS_encoded, testDS_encoded, trainLabels_encoded, testLabels_encoded = \
    cross_validation.train_test_split(allDS_encoded, allLabels_encoded, test_size = 0.4, random_state = 0 )

### 3.2 Input Normalization

#### 3.2.1 Training Data Set

In [ ]:
scaler = pp.MinMaxScaler().fit(trainDS_encoded)

In [ ]:
trainDS_scaled = pd.DataFrame(scaler.transform(trainDS_encoded), columns =  continuousCols)

In [ ]:
trainDS_scaled.describe()

#### 3.2.2 Test Data Set

**WARNING**: Using the scaler from *trainDS*

In [ ]:
testDS_scaled = pd.DataFrame(scaler.transform(testDS_encoded), columns =  continuousCols)

In [ ]:
testDS_scaled.describe()

### 3.3 Principal Component Analysis (PCA)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_features = trainDS_scaled.columns.size

In [ ]:
print "Total number of features: %d" %n_features

In [ ]:
pca = PCA(n_components=n_features, whiten=False)
pca.fit(trainDS_scaled)

In [ ]:
#accum explained variance ration
pca.explained_variance_ratio_[0:].cumsum()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(1 - pca.explained_variance_ratio_.cumsum(), drawstyle = 'steps-post')
plt.title('PCA Reconstruction Error');

In [ ]:
n_factors = sum(1-pca.explained_variance_ratio_[0:].cumsum() > 0.10)
print "Number of factors with 10% of reonstraction Error: ", n_factors

In [ ]:
pca = PCA(n_components=n_factors)
pca.fit(trainDS_scaled)

In [ ]:
print "Explained Variance Ratio"
sum(pca.explained_variance_ratio_)

In [ ]:
trainDS_pca = pca.transform(trainDS_scaled)

**WARNING**: Using the pca from *trainDS_scaled* to *testDS_scaled*

In [ ]:
testDS_pca = pca.transform(testDS_scaled)

-----

## 4. Modeling

In [ ]:
from sklearn import metrics

## 4.1 Decision Trees

* **Parameters**: 
    * *criterion*: The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain
    * *max_depth*: The maximum depth of the tree.
    * *min_samples_leaf* : The minimum number of samples required to be at a leaf node.
* **Scalability**:	The problem of learning an optimal decision tree is known to be NP-complete under several aspects of optimality and even for simple concepts. Consequently, practical decision-tree learning algorithms are based on heuristic algorithms such as the greedy algorithm where locally optimal decisions are made at each node. Such algorithms cannot guarantee to return the globally optimal decision tree
* **Usecase**:	General-purpose
* **Geometry (metric used)**: Gini impurity and entropy
* **Observations:** 
    * Requires little data preparation. Other techniques often require data normalisation, dummy variables need to be created and blank values to be removed. Note however that this module does not support missing values.
    * Consider performing dimensionality reduction (PCA, ICA, or Feature selection) beforehand to give your tree a better chance of finding features that are discriminative.
    * Able to handle both numerical and categorical data. Other techniques are usually specialised in analysing datasets that have only one type of variable
		 	

In [ ]:
from sklearn import tree

In [ ]:
clt = tree.DecisionTreeClassifier(criterion='gini')
%time clt.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Tree depth: ", clt.tree_.max_depth

In [ ]:
print "Mean accuracy on the given test data and labels: ", clt.score(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: ", clt.score(testDS_pca, testLabels_encoded)

In [ ]:
lb_all_attack_class.inverse_transform(clt.predict(testDS_pca))

### 4.1.1 Find the best tree depth (Gini impurity)

In [ ]:
def getDecisionTreeMesures(initArg = 'gini', max_depth = 2, 
                           train_labels = None, train_data = None, 
                           test_label = None, test_data = None):
    model = tree.DecisionTreeClassifier(criterion=initArg, max_depth=max_depth, 
                                        min_samples_leaf=500)
    model.fit(train_data, train_labels)
    return [model.tree_.max_depth,
            model.score(train_data, train_labels), #E_in 
            model.score(test_data, test_label)] #E_out

In [ ]:
gini_measures = np.array([getDecisionTreeMesures('gini', max_depth, trainLabels_encoded, trainDS_pca, 
                                            testLabels_encoded, testDS_pca)
                     for max_depth in range(10,1, -1)])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14, 5) )
ax1, ax2 = axes.ravel()

#ax1.figure(figsize=(7,5))
ax1.plot(gini_measures[:,0], gini_measures[:,1], label = 'Score in', c = 'b')
ax1.legend(loc=4)
ax1.set_ylim(0.8,0.95)
ax1.set_title('Decision Tree Scores (Gini Criterion) E_in')
ax1.set_xlabel("Max Depth of Tree")
ax1.set_ylabel("Scores In");

#ax2.figure(figsize=(7,5))
ax2.plot(gini_measures[:,0], gini_measures[:,2], label = 'Score out', c='g')
ax2.legend(loc=4)
ax2.set_ylim(0.8,0.95)
ax2.set_title("Decision Tree Scores (Gini Criterion)")
ax2.set_xlabel("Max Depth of Tree")
ax2.set_ylabel("Scores Out");

#### Gini Decision Tree with max_dept = 5

In [ ]:
clt = tree.DecisionTreeClassifier(criterion='gini', max_depth=5, min_samples_leaf=500)
%time clt.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: ", clt.score(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: ", clt.score(testDS_pca, testLabels_encoded)

In [ ]:
lb_all_attack_class.inverse_transform(clt.predict(testDS_pca))

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.core.display import Image
import pydot 

In [ ]:
#Visualize the decision tree
dot_data = StringIO() 
tree.export_graphviz(clt, out_file=dot_data, feature_names=trainDS_encoded.columns) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
Image(graph.create_png())

**Exercice 1**: Make the same study but with the criterion of Entropy

## 4.2 Random Forest

* **Parameters**: 
    * *n_estimators*: The number of trees in the forest.
    * *criterion*: The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain
    * *min_samples_leaf* : The minimum number of samples required to be at a leaf node.
    * *max_features* : The number of features to consider when looking for the best split
* **Scalability**:	Similar to Decision Tree
* **Usecase**:	General-purpose
* **Geometry (metric used)**: Gini impurity and entropy
		 	

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=10, min_samples_leaf=50)
%time clf.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given train data and labels: ", clf.score(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: ", clf.score(testDS_pca, testLabels_encoded)

In [ ]:
test_predicted = pd.DataFrame(lb_all_attack_class.inverse_transform(clf.predict(testDS_pca)), 
                              columns=["class_attack"], dtype='category')

In [ ]:
test_predicted.groupby("class_attack").size()

### 4.2.1 Find the best min_samples_leaf

In [ ]:
def getRFMesures(initArg = 'gini', min_samples_leaf = 50, num_estimators = 10, max_features = 1,
                           train_labels = None, train_data = None, 
                           test_label = None, test_data = None):
    model = RandomForestClassifier(n_estimators = num_estimators, 
                                   max_features = max_features,
                                   min_samples_leaf= min_samples_leaf)
    model.fit(train_data, train_labels)
    return [model.max_features,
            model.score(train_data, train_labels), #E_in 
            model.score(test_data, test_label)] #E_out

In [ ]:
RF_measures = np.array([getRFMesures('gini', 50, 10, max_features,
                                     trainLabels_encoded, trainDS_pca, 
                                     testLabels_encoded, testDS_pca)
                     for max_features in range(2, 16, 1)])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(18, 5) )
ax1, ax2 = axes.ravel()

ax1.plot(RF_measures[:,0], RF_measures[:,1], label = 'Score in', c = 'b')
ax1.legend(loc=4)
ax1.set_ylim(0.9365,0.939)
ax1.set_title('Random Forest Scores. E_in')
ax1.set_xlabel("Max num of features")
ax1.set_ylabel("Scores In");

#ax2.figure(figsize=(7,5))
ax2.plot(RF_measures[:,0], RF_measures[:,2], label = 'Score out', c='g')
ax2.legend(loc=4)
ax2.set_ylim(0.930,0.936)
ax2.set_title("Random Forest Scores. E_out")
ax2.set_xlabel("Max num of features")
ax2.set_ylabel("Scores Out");

#### Entropy Decision Tree with max_num_features = 13

In [ ]:
clf = RandomForestClassifier(n_estimators = 13, min_samples_leaf = 50, max_features = 10)
%time clf.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given train data and labels: "
clf.score(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: "
clf.score(testDS_pca, testLabels_encoded)

------

## 4.2 Logistic Regression

* **Parameters**: 
   * *C* : Inverse of regularization strength; must be a positive float. Smaller values specify stronger regularization.
* **Usecase**:	Clasification
* **Geometry (metric used)**: l1 and l2, used to specify the norm used in the penalization
		 	

In [ ]:
from sklearn import linear_model

### 4.2.1 Logistic Regression with all class attack

In [ ]:
trainDS_encoded, testDS_encoded, trainLabels_encoded, testLabels_encoded = \
    cross_validation.train_test_split(allDS_encoded, allLabels.class_attack, test_size = 0.4, random_state = 0 )

In [ ]:
logreg = linear_model.LogisticRegression(C=1e5)
%time logreg.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given train data and labels: "
logreg.score(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given test data and labels: "
logreg.score(testDS_pca, testLabels_encoded)

In [ ]:
predicted = logreg.predict(testDS_pca)

In [ ]:
cm = pd.DataFrame(metrics.confusion_matrix(testLabels_encoded, predicted, attack_class), 
                  columns=attack_class, index=attack_class)

In [ ]:
print "Predicted 'normal' vs true values:"
cm.ix["normal"]

In [ ]:
print "True 'normal' vs predicted :"
cm["normal"]

#### 4.2.1.1 Finding out the best regularization parameter

In [ ]:
def getLRMesures(C = 1e5, train_labels = None, train_data = None, 
                 test_labels = None, test_data = None):
    model = linear_model.LogisticRegression(C = C)
    model.fit(train_data, train_labels)
    return [model.C,
            model.score(train_data, train_labels), #E_in 
            model.score(test_data, test_labels)] #E_out

In [ ]:
Cs = [1e5, 1e4, 1e3, 1e2, 1e1, 1e0, 1e-1 ]

In [ ]:
LR_measures = np.array([getLRMesures(C = c, 
                                     train_labels=trainLabels_encoded, 
                                     train_data = trainDS_pca, 
                                     test_labels = testLabels_encoded, 
                                     test_data = testDS_pca) 
                        for c in Cs])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14, 5) )
ax1, ax2 = axes.ravel()

ax1.plot(LR_measures[:,0], LR_measures[:,1], label = 'Score in', c = 'b')
ax1.legend(loc=4)
ax1.set_title('Logistic Regresion Scores. E_in')
ax1.set_xscale("log")
ax1.set_xlabel("Inverse of regularization strength (log)")
ax1.set_ylabel("Scores In")
ax1.set_ylim(0.925,0.932);

#ax2.figure(figsize=(7,5))
ax2.plot(LR_measures[:,0], LR_measures[:,2], label = 'Score out', c='g')
ax2.legend(loc=4)
ax2.set_title("Logistic Regresion Scores. E_out")
ax2.set_xscale("log")
ax2.set_xlabel("Inverse of regularization strength (log)")
ax2.set_ylabel("Scores Out")
ax2.set_ylim(0.92,0.93);


#### 4.2.1.2 Finding out the best regularization parameter with Cross Validation

In [ ]:
from sklearn.cross_validation import StratifiedKFold

In [ ]:
skf = StratifiedKFold(trainLabels_encoded, len(Cs))

In [ ]:
trainLabels_encoded.describe()

In [ ]:
LR_measures_CV = np.array([getLRMesures(C = c, 
                                        train_labels = trainLabels_encoded[train_ix], 
                                        train_data = trainDS_pca[train_ix, :], 
                                        test_labels = trainLabels_encoded[cv_ix], 
                                        test_data = trainDS_pca[cv_ix, :]) 
                            for c, (train_ix, cv_ix) in zip(Cs,skf)])

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14, 5) )
ax1, ax2 = axes.ravel()

ax1.plot(LR_measures_CV[:,0], LR_measures_CV[:,1], label = 'Score in', c = 'b')
ax1.legend(loc=4)
ax1.set_title('Logistic Regresion Scores with Cross Validation. E_in')
ax1.set_xscale("log")
ax1.set_xlabel("Inverse of regularization strength (log)")
ax1.set_ylabel("Scores In")
ax1.set_ylim(0.925,0.932);

#ax2.figure(figsize=(7,5))
ax2.plot(LR_measures_CV[:,0], LR_measures_CV[:,2], label = 'Score out', c='g')
ax2.legend(loc=4)
ax2.set_title("Logistic Regresion Scores with Cross Validation. E_out")
ax2.set_xscale("log")
ax2.set_xlabel("Inverse of regularization strength (log)")
ax2.set_ylabel("Scores Out")
ax2.set_ylim(0.925,0.935);


*The Inverse regularization strength* **C =  1000**

New model with C = 1000

In [ ]:
#logreg_cv = linear_model.LogisticRegression(C = 10)
#%time logreg_cv.fit(trainDS_pca[skf.test_folds != 3], trainLabels.class_attack[skf.test_folds != 3])

In [ ]:
logreg_cv = linear_model.LogisticRegression(C = 1000)
%time logreg_cv.fit(trainDS_pca, trainLabels_encoded)

In [ ]:
print "Mean accuracy on the given train Cross Validation data and labels: ", 
logreg_cv.score(trainDS_pca[skf.test_folds != 3], 
                trainLabels_encoded[skf.test_folds != 3])

In [ ]:
print "Mean accuracy on the given train Cross Validation data and labels: ", 
logreg_cv.score(trainDS_pca[skf.test_folds == 3], 
                trainLabels_encoded[skf.test_folds == 3])

In [ ]:
print "Mean accuracy on the given test data and labels: ", 
logreg_cv.score(testDS_pca, testLabels_encoded)

**Exercice 2**: Make the same study but with is_normal label